# Спрос на рынке жилья (за счет ипотеки)

In [229]:
from IPython.display import Markdown as md
import pandas as pd
import numpy as np

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf

from scipy.integrate import quad
from scipy.stats import rv_continuous, norm, gmean

%run ../COMMON/common.ipynb # загрузка общих функций и констант, все, что оттуда, должно иметь префикс common.

conWork = sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=common.strYearDBPath)) # connection к рабочей базе данных
conWorkEx = sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=common.strExYearDBPath)) # connection к к рабочей базе данных экзогенных переменных
conWorkExH=sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=common.strExParamDBPath)) # connection к рабочей базе данных экзогенных параметров
conWorkSvod=sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=common.strSvodDBPath)) # connection к рабочей базе данных СВОД


In [230]:
pd.set_option('display.max_columns',None)

## Установка временных точек

In [231]:
iFirstFactYear=2005
iLastFactYear=2019

iFirstForecastYear=iLastFactYear+1
iLastForecastYear=2030

## Загрузка исходных данных

Исходные данные:
  - Фактические значения - из базы данных `year.sqlite3`;
  - Экзогенные - из базы данных `exog_year.sqlite3`
  - Задаваемые вручную - из базы данных `exog_param.sqlite3`
  - Результаты других моделей - из базы данных `svod.sqlite3`. *В эту же базу записываем результаты данного блокнота*. А даном блокноте используются результаты блока **"Цены и себестоимость"**

In [232]:
class demand_ipot:
    pdfWork=None
    
    lstYearCodes=['ZPN_I','ZPR_I','Unmpl_s','Inc_x','MEPop','DispPop','GDP_x','CPIAv','KeyRate',
                  'HHAv','Pop_x','p_MortgLifeAv_x','p_ProbDef','ConstrPriceIZD_x','p_superrich',
                  'loans_and_ref_vol_MKD_vtor','ipot_debt','ipot_debt_prava','loans_and_ref_vol_MKD',
                  'sdelkiddy_ipot_x','sdelkikp_ipot_x','portf_rate','portf_term',
                  'loans_vol_MKD_perv','refinance_vol','ICB','loan_rate','outstanding_debt_s',
                  'loan_vol_av','UZUipot_x','ZhilFondunits','ZhilFondIZDunits','CPR','Activ_x',
                  'excess_of_flat_cost','s_ipot_inIZDVvody','s_ipot_inFPVvody'] # список экзогенных и фактических данных
 
    lstSvod=['price1mddy_alt_x','price1mall_alt_x','price1mall_x','ZhilFond','ZhilFondIZD','VvodyMKDunits',
             'AvSqIZD','VvodyIZD','AddIZD_x','AvSqVv','AvSqDdy','AvSqVtor'] # результаты из блока "Цены и себестоимость"

    def __init__(self):
        assert False, 'you can\'t create variables of demand_om class!'
    
    def MakeWorkFrame():
        ''' Загрузка исходных данных
            Исходные данные:
                Фактические значения - из базы данных year.sqlite3;
                Экзогенные - из базы данных exog_year.sqlite3
                Задаваемые вручную - из базы данных exog_param.sqlite3
                Результаты других моделей - из базы данных svod.sqlite3. 
        '''
        
        pdfAct=(pd.read_sql(common.make_SELECT_YEAR_string(demand_ipot.lstYearCodes), con=conWork)
        .pipe(common.make_frame)
        .pipe(common.scale, list_fields=['KeyRate','loan_rate','CPR'], multiplier=1e2)
        .pipe(common.scale, list_fields=['Inc_x'], multiplier=1e3)
        .pipe(common.scale, list_fields=['Activ_x'], multiplier=1e-3)
        .pipe(common.scale, list_fields=['sdelkiddy_ipot_x','loans_vol_MKD_perv','refinance_vol'], multiplier=1e-6))
        
        pdfExog=(pd.read_sql(common.make_SELECT_YEAR_string(demand_ipot.lstYearCodes), con=conWorkEx)
        .pipe(common.make_frame)
        .pipe(common.scale, list_fields=['GDP_x'], multiplier=1e-3))
        
        PdfExogHandle=(pd.read_sql(common.make_SELECT_YEAR_string(demand_ipot.lstYearCodes), con=conWorkExH)
        .pipe(common.make_frame))

        pdfSvod=(pd.read_sql(common.make_SELECT_YEAR_string(demand_ipot.lstSvod), con=conWorkSvod)
        .pipe(common.make_frame)
        .pipe(common.scale, list_fields=['price1mddy_alt_x', 'price1mall_alt_x'], multiplier=1e3))

        demand_ipot.pdfWork=(pdfAct.combine_first(pdfExog).combine_first(PdfExogHandle).combine_first(pdfSvod)
        .pipe(common.scale, list_fields=['ZPR_I','Unmpl_s'], multiplier=1e2)
        .pipe(common.add, list_fields=['ZPR_I'], param=-100)
        .pipe(common.scale, list_fields=['Pop_x'], multiplier=1e3))
        
        demand_ipot.pdfWork['Inc_agg_x'] =  demand_ipot.pdfWork['Inc_x']*demand_ipot.pdfWork['Pop_x']*12/1000
        
        return demand_ipot.pdfWork
    
# a=demand_om()


In [233]:
demand_ipot.MakeWorkFrame()

,Activ_x,AddIZD_x,AvSqDdy,AvSqIZD,AvSqVtor,AvSqVv,CPIAv,CPR,ConstrPriceIZD_x,DispPop,GDP_x,HHAv,ICB,Inc_x,KeyRate,MEPop,Pop_x,UZUipot_x,Unmpl_s,VvodyIZD,VvodyMKDunits,ZPN_I,ZPR_I,ZhilFond,ZhilFondIZD,ZhilFondIZDunits,ZhilFondunits,excess_of_flat_cost,ipot_debt,ipot_debt_prava,loan_rate,loan_vol_av,loans_and_ref_vol_MKD,loans_and_ref_vol_MKD_vtor,loans_vol_MKD_perv,outstanding_debt_s,p_MortgLifeAv_x,p_ProbDef,p_superrich,portf_rate,portf_term,price1mall_alt_x,price1mall_x,price1mddy_alt_x,refinance_vol,s_ipot_inFPVvody,s_ipot_inIZDVvody,sdelkiddy_ipot_x,sdelkikp_ipot_x,Inc_agg_x
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2004,7.137613,NaN,NaN,NaN,NaN,NaN,1.108812,NaN,NaN,NaN,17.027191,2.677054,NaN,6410.300000,13.520000,NaN,144067.300,NaN,7.714864,NaN,NaN,1.225698,10.6000,NaN,NaN,15.725665,56.854367,NaN,0.015000,NaN,NaN,NaN,0.018500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.108218e+07
2005,9.750847,NaN,65.703978,138.031496,NaN,67.268041,1.127055,NaN,NaN,0.756100,21.609765,2.661470,NaN,8088.300000,12.980000,8.711611,143518.814,NaN,7.153243,17.530000,0.388000,1.269288,12.6000,2955.700000,NaN,15.670000,57.402292,NaN,0.052800,NaN,14.900000,NaN,0.056341,NaN,NaN,0.210113,14.550000,NaN,0.02,NaN,NaN,21048.567265,22.245801,18491.692819,NaN,NaN,NaN,0.032140,0.109063,1.392988e+07
2006,14.046610,NaN,65.107090,138.672222,NaN,65.806452,1.096830,NaN,NaN,0.769907,26.917201,2.645886,NaN,10154.800000,8.894216,8.928732,143049.637,NaN,7.062219,19.968800,0.465000,1.243016,13.3000,3003.000000,NaN,NaN,NaN,NaN,0.233897,NaN,13.700000,NaN,0.263561,NaN,NaN,0.042982,NaN,NaN,0.02,NaN,NaN,28597.165335,31.087628,24937.896936,NaN,NaN,NaN,0.043254,0.146776,1.743169e+07
2007,20.125188,NaN,64.216907,138.458421,NaN,65.601504,1.089881,NaN,NaN,0.785663,33.247513,2.630302,NaN,12540.200000,7.512557,9.129498,142805.114,NaN,6.029393,26.307100,0.532000,1.278258,17.2000,3060.000000,NaN,NaN,NaN,NaN,0.611222,NaN,12.600000,NaN,0.556489,NaN,NaN,0.272760,NaN,0.10,0.02,NaN,NaN,37938.493305,44.119095,32765.902784,NaN,NaN,NaN,0.058210,0.197530,2.148966e+07
2008,28.022340,NaN,63.243902,136.835000,NaN,64.612676,1.141048,11.127132,NaN,0.782878,41.276849,2.614717,NaN,14863.600000,8.129107,9.301331,142742.366,0.684561,6.241192,27.367000,0.568000,1.271978,11.5000,3116.000000,NaN,NaN,NaN,1.50,1.070329,0.094510,12.900000,1.876407,0.655808,534808.0,0.121000,1.673344,17.941667,0.10,0.02,12.900000,17.941667,44576.706072,53.253399,37788.915681,0.000000,NaN,NaN,0.059157,0.285017,2.545999e+07
2009,29.430000,NaN,61.675214,135.934000,50.877922,63.821138,1.116977,11.014075,NaN,0.783533,38.807219,2.599133,NaN,16895.000000,11.310000,9.427587,142785.348,0.957832,8.252236,28.546140,0.492000,1.077964,-3.5000,3176.600000,974.270500,16.261078,59.547624,1.50,1.010889,0.109001,14.300000,1.172318,0.152501,136501.0,0.016000,4.701409,16.458333,0.10,0.02,13.178967,16.845357,42565.501231,51.857276,36473.861415,0.000000,NaN,NaN,0.012520,0.116437,2.894830e+07
2010,33.804600,NaN,60.086697,132.907292,50.985166,62.666667,1.068508,10.625308,NaN,0.782280,46.308541,2.583549,NaN,18958.400000,8.030000,9.546797,142849.468,1.340190,7.344543,25.518200,0.525000,1.124155,5.2000,3229.000000,1000.360000,16.415168,60.126706,1.50,1.129400,0.109108,13.100000,1.260847,0.380061,341061.0,0.039000,5.699277,16.358333,0.10,0.02,13.161161,16.026515,41870.584649,53.768465,35853.805771,0.000313,NaN,NaN,0.028376,0.270253,3.249837e+07
2011,41.627500,NaN,58.132834,133.139303,51.084428,60.683761,1.084634,12.224180,NaN,0.775441,60.114001,2.544156,NaN,20780.000000,8.120000,9.644999,142960.908,1.875183,6.495643,26.761000,0.585000,1.115359,2.8000,3286.153000,1027.764400,16.595561,60.804505,1.50,1.478980,0.077474,11.900000,1.369306,0.716944,608944.0,0.108000,3.857936,14.908333,0.10,0.02,12.612582,14.980855,44815.617500,44.815618,37897.472700,0.000841,NaN,NaN,0.072355,0.444096,3.564873e+07
2012,49.509600,NaN,56.425400,134.490995,51.185577,59.489633,1.050667,11.408883,NaN,0.781314,68.103450,2.504763,NaN,23221.1000

### модель №2 "Доля просроченной задолженности по ипотеке"

In [234]:
pdfXY2 = demand_ipot.pdfWork.loc[iFirstFactYear:iLastFactYear,['outstanding_debt_s','Unmpl_s','ZPR_I']].dropna()
resDEBTX = smf.ols(formula='outstanding_debt_s ~ outstanding_debt_s.shift(1) + Unmpl_s + ZPR_I', missing='drop', 
                   data=pdfXY2.loc[2005:2018]).fit()
print(resDEBTX.summary())

                            OLS Regression Results                            
Dep. Variable:     outstanding_debt_s   R-squared:                       0.842
Model:                            OLS   Adj. R-squared:                  0.789
Method:                 Least Squares   F-statistic:                     15.95
Date:                Thu, 04 Mar 2021   Prob (F-statistic):           0.000603
Time:                        16:34:01   Log-Likelihood:                -11.899
No. Observations:                  13   AIC:                             31.80
Df Residuals:                       9   BIC:                             34.06
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept         

In [235]:
for i in range (iFirstForecastYear,iLastForecastYear+1):
    demand_ipot.pdfWork.loc[i,'outstanding_debt_s']=(resDEBTX.params[0]
                                                    +resDEBTX.params[1]*demand_ipot.pdfWork.loc[i-1,'outstanding_debt_s']
                                                    +resDEBTX.params[2]*demand_ipot.pdfWork.loc[i,'Unmpl_s']
                                                    +resDEBTX.params[3]*demand_ipot.pdfWork.loc[i,'ZPR_I'])

### модель №1 "Средняя ставка по выданным кредитам"

In [236]:
pdfXY = demand_ipot.pdfWork.loc[iFirstFactYear:iLastFactYear, ['loan_rate', 'KeyRate', 'outstanding_debt_s']].dropna()
resLORAX = smf.ols(formula='loan_rate ~ KeyRate + outstanding_debt_s - 1', missing='drop', 
                   data=pdfXY.loc[2008:2019]).fit()
print(resLORAX.summary())

                                 OLS Regression Results                                
Dep. Variable:              loan_rate   R-squared (uncentered):                   0.980
Model:                            OLS   Adj. R-squared (uncentered):              0.976
Method:                 Least Squares   F-statistic:                              242.3
Date:                Thu, 04 Mar 2021   Prob (F-statistic):                    3.38e-09
Time:                        16:34:01   Log-Likelihood:                         -23.620
No. Observations:                  12   AIC:                                      51.24
Df Residuals:                      10   BIC:                                      52.21
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                         coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------

In [237]:
ostatok = demand_ipot.pdfWork.loc[iLastFactYear,'loan_rate']-resLORAX.predict(demand_ipot.pdfWork).loc[iLastFactYear]
demand_ipot.pdfWork.loc[iFirstForecastYear:iLastForecastYear,'loan_rate']=resLORAX.predict(demand_ipot.pdfWork).loc[iFirstForecastYear:iLastForecastYear]+ostatok

### модель №3 "Средний размер ипотечного кредита"

In [238]:
demand_ipot.pdfWork['D15']=0
demand_ipot.pdfWork.loc[2015,'D15']=1

demand_ipot.pdfWork['flat_price']=((demand_ipot.pdfWork['AvSqVv']*demand_ipot.pdfWork['price1mall_x'])/(demand_ipot.pdfWork['AvSqVv'].shift(1)*demand_ipot.pdfWork['price1mall_x'].shift(1))-1)*100
demand_ipot.pdfWork['ZPN_rI']=(demand_ipot.pdfWork['ZPN_I']-1)*100
demand_ipot.pdfWork['loan_vol_av_rI']=(demand_ipot.pdfWork['loan_vol_av']/demand_ipot.pdfWork['loan_vol_av'].shift(1)-1)*100

pdfXY3 = demand_ipot.pdfWork.loc[iFirstFactYear:iLastFactYear,['loan_vol_av_rI','flat_price','ZPN_rI','D15']].dropna()
resLOVORI = smf.ols(formula='loan_vol_av_rI ~ flat_price.shift(1) + ZPN_rI + D15 -1', missing='drop', 
                    data=pdfXY3.loc[2009:2016]).fit()
print(resLOVORI.summary())

                                 OLS Regression Results                                
Dep. Variable:         loan_vol_av_rI   R-squared (uncentered):                   0.982
Model:                            OLS   Adj. R-squared (uncentered):              0.968
Method:                 Least Squares   F-statistic:                              71.37
Date:                Thu, 04 Mar 2021   Prob (F-statistic):                    0.000627
Time:                        16:34:01   Log-Likelihood:                         -9.9323
No. Observations:                   7   AIC:                                      25.86
Df Residuals:                       4   BIC:                                      25.70
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                          coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------

In [239]:
demand_ipot.pdfWork.loc[iFirstForecastYear:iLastForecastYear,'loan_vol_av_rI']=resLOVORI.predict(demand_ipot.pdfWork).loc[iFirstForecastYear:iLastForecastYear]

for i in range (iFirstForecastYear,iLastForecastYear+1):
    demand_ipot.pdfWork.loc[i,'loan_vol_av']=demand_ipot.pdfWork.loc[i-1,'loan_vol_av']*(demand_ipot.pdfWork.loc[i,'loan_vol_av_rI']/100+1)

### модель №4 "Площадь жилья, приобретенного за счет ипотеки на рынке строящегося жилья"

In [240]:
demand_ipot.pdfWork.loc[iFirstFactYear:iLastFactYear,'squareddy_ipot_x']=demand_ipot.pdfWork.loc[iFirstFactYear:iLastFactYear,'AvSqDdy']*demand_ipot.pdfWork.loc[iFirstFactYear:iLastFactYear,'sdelkiddy_ipot_x']

#### доп рассчеты для модели 4 (Строящееся)

In [241]:
demand_ipot.pdfWork['month_pay_perv']=np.pmt(demand_ipot.pdfWork['loan_rate']/12/100,demand_ipot.pdfWork['p_MortgLifeAv_x']*12,-demand_ipot.pdfWork['price1mddy_alt_x']*54*0.7)/1000
demand_ipot.pdfWork['month_pay_all']=np.pmt(demand_ipot.pdfWork['loan_rate']/12/100,demand_ipot.pdfWork['p_MortgLifeAv_x']*12,-demand_ipot.pdfWork['price1mall_x']*1000*54*0.7)/1000
demand_ipot.pdfWork['mininc_for_loan_approval_perv']=demand_ipot.pdfWork['month_pay_perv']*1000/0.35/demand_ipot.pdfWork['HHAv']

In [242]:
def _calc_perca_income(x):
    return common.kolmakov.cdf(x['mininc_for_loan_approval_perv'], sigma=x['DispPop'], mu=x['MEPop'])

demand_ipot.pdfWork['_income_percapita_perv']=demand_ipot.pdfWork[['mininc_for_loan_approval_perv', 'DispPop', 
                                              'MEPop']].apply(_calc_perca_income, axis=1)

demand_ipot.pdfWork['_s_income_available_ipot_perv']=(demand_ipot.pdfWork['Inc_agg_x']-12*demand_ipot.pdfWork['_income_percapita_perv']*demand_ipot.pdfWork['Pop_x']/1000)/demand_ipot.pdfWork['Inc_agg_x']
demand_ipot.pdfWork['poten_cashpay_4loan_perv']=(demand_ipot.pdfWork['Inc_agg_x']-12*demand_ipot.pdfWork['_income_percapita_perv']*demand_ipot.pdfWork['Pop_x']/1000)*0.35
demand_ipot.pdfWork['poten_loans_vol_perv']=-np.pv(demand_ipot.pdfWork['loan_rate']/100,demand_ipot.pdfWork['p_MortgLifeAv_x'],demand_ipot.pdfWork['poten_cashpay_4loan_perv'])

demand_ipot.pdfWork['_inc_percapita_superrich']=np.exp(norm.ppf(1-demand_ipot.pdfWork['p_superrich'], 
                                                                loc=demand_ipot.pdfWork['MEPop'], 
                                                                scale=demand_ipot.pdfWork['DispPop']))

def _calc_superrich_perca_income(x):
    return 1-common.kolmakov.cdf(x['_inc_percapita_superrich'], sigma=x['DispPop'], mu=x['MEPop'])/x['Inc_x']

demand_ipot.pdfWork['_p_income_superr']=demand_ipot.pdfWork[['_inc_percapita_superrich', 'DispPop', 
                                        'MEPop', 'Inc_x']].apply(_calc_superrich_perca_income, axis=1)

demand_ipot.pdfWork['poten_cashpay_4loan_exsuperrich_perv']=demand_ipot.pdfWork['Inc_agg_x']*(demand_ipot.pdfWork['_s_income_available_ipot_perv']-demand_ipot.pdfWork['_p_income_superr'])*0.35+(12*demand_ipot.pdfWork['mininc_for_loan_approval_perv']*demand_ipot.pdfWork['p_superrich']*demand_ipot.pdfWork['excess_of_flat_cost']*demand_ipot.pdfWork['Pop_x'])/1000
demand_ipot.pdfWork['poten_loans_vol_exsuperrich_perv']=-np.pv(demand_ipot.pdfWork['loan_rate']/100,demand_ipot.pdfWork['p_MortgLifeAv_x'],demand_ipot.pdfWork['poten_cashpay_4loan_exsuperrich_perv'])

demand_ipot.pdfWork['poten_loans_2income_perv']=demand_ipot.pdfWork['poten_loans_vol_exsuperrich_perv']/demand_ipot.pdfWork['Inc_agg_x']

demand_ipot.pdfWork['_s_family_available_ipot']=1-norm.cdf(np.log(demand_ipot.pdfWork['month_pay_all']*1000/0.35/demand_ipot.pdfWork['HHAv']), 
                                                           loc=demand_ipot.pdfWork['MEPop'], 
                                                           scale=demand_ipot.pdfWork['DispPop'])

demand_ipot.pdfWork['UZUipot_s']=demand_ipot.pdfWork['UZUipot_x']/((demand_ipot.pdfWork['Pop_x']/demand_ipot.pdfWork['HHAv'])/1000)
demand_ipot.pdfWork.loc[2007,'UZUipot_s']=demand_ipot.pdfWork.loc[2008,'UZUipot_s']*demand_ipot.pdfWork.loc[2008,'UZUipot_s']/demand_ipot.pdfWork.loc[2009,'UZUipot_s']

demand_ipot.pdfWork['poten_loans_2income_UZU_corrected_perv']=demand_ipot.pdfWork['poten_loans_2income_perv']*(1-demand_ipot.pdfWork['UZUipot_s'].shift(1)/demand_ipot.pdfWork['_s_family_available_ipot'])

### модель №5 "Площадь жилья, приобретенного за счет ипотеки на рынке готового жилья"

In [243]:
demand_ipot.pdfWork['squarekp_ipot_x']=demand_ipot.pdfWork['sdelkikp_ipot_x']*demand_ipot.pdfWork['AvSqVtor']
demand_ipot.pdfWork.loc[2008,'squarekp_ipot_x']=14.7638835130233

#### доп рассчеты для модели 5

In [244]:
demand_ipot.pdfWork['month_pay_vtor']=np.pmt(demand_ipot.pdfWork['loan_rate']/12/100,demand_ipot.pdfWork['p_MortgLifeAv_x']*12,-demand_ipot.pdfWork['price1mall_alt_x']*54*0.7)/1000
demand_ipot.pdfWork['mininc_for_loan_approval_vtor']=demand_ipot.pdfWork['month_pay_vtor']*1000/0.35/demand_ipot.pdfWork['HHAv']

def _calc_perca_income(x):
    return common.kolmakov.cdf(x['mininc_for_loan_approval_vtor'], sigma=x['DispPop'], mu=x['MEPop'])

demand_ipot.pdfWork['_income_percapita_vtor']=demand_ipot.pdfWork[['mininc_for_loan_approval_vtor', 'DispPop', 
                                              'MEPop']].apply(_calc_perca_income, axis=1)

demand_ipot.pdfWork['_s_income_available_ipot_vtor']=(demand_ipot.pdfWork['Inc_agg_x']-12*demand_ipot.pdfWork['_income_percapita_vtor']*demand_ipot.pdfWork['Pop_x']/1000)/demand_ipot.pdfWork['Inc_agg_x']
demand_ipot.pdfWork['poten_cashpay_4loan_vtor']=(demand_ipot.pdfWork['Inc_agg_x']-12*demand_ipot.pdfWork['_income_percapita_vtor']*demand_ipot.pdfWork['Pop_x']/1000)*0.35
demand_ipot.pdfWork['poten_loans_vol_vtor']=-np.pv(demand_ipot.pdfWork['loan_rate']/100,demand_ipot.pdfWork['p_MortgLifeAv_x'],demand_ipot.pdfWork['poten_cashpay_4loan_vtor'])

demand_ipot.pdfWork['poten_cashpay_4loan_exsuperrich_vtor']=demand_ipot.pdfWork['Inc_agg_x']*(demand_ipot.pdfWork['_s_income_available_ipot_vtor']-demand_ipot.pdfWork['_p_income_superr'])*0.35+(12*demand_ipot.pdfWork['mininc_for_loan_approval_vtor']*demand_ipot.pdfWork['p_superrich']*demand_ipot.pdfWork['excess_of_flat_cost']*demand_ipot.pdfWork['Pop_x'])/1000
demand_ipot.pdfWork['poten_loans_vol_exsuperrich_vtor']=-np.pv(demand_ipot.pdfWork['loan_rate']/100,demand_ipot.pdfWork['p_MortgLifeAv_x'],demand_ipot.pdfWork['poten_cashpay_4loan_exsuperrich_vtor'])

demand_ipot.pdfWork['poten_loans_2income_vtor']=demand_ipot.pdfWork['poten_loans_vol_exsuperrich_vtor']/demand_ipot.pdfWork['Inc_agg_x']

demand_ipot.pdfWork['poten_loans_2income_UZU_corrected_vtor']=demand_ipot.pdfWork['poten_loans_2income_vtor']*(1-demand_ipot.pdfWork['UZUipot_s'].shift(1)/demand_ipot.pdfWork['_s_family_available_ipot'])

### Оценивание моделей 4 и 5

In [245]:
##### объясняющие переменные для модели 5

demand_ipot.pdfWork['poten_loans_2income_UZU_corrected_vtor_']=demand_ipot.pdfWork['poten_loans_2income_UZU_corrected_vtor']*100
demand_ipot.pdfWork['ddyprice_def_corrected_2kp']=(1+demand_ipot.pdfWork['p_ProbDef'])*demand_ipot.pdfWork['price1mddy_alt_x']/demand_ipot.pdfWork['price1mall_alt_x']*100

##### объясняющие переменные для модели 4

demand_ipot.pdfWork['loans2income_2pricesratio']=demand_ipot.pdfWork['poten_loans_2income_UZU_corrected_perv']*10000/demand_ipot.pdfWork['ddyprice_def_corrected_2kp']

#### Модель 5

In [246]:
pdfXY5 = demand_ipot.pdfWork.loc[iFirstFactYear:iLastFactYear, ['squarekp_ipot_x','Unmpl_s','poten_loans_2income_UZU_corrected_vtor_','ddyprice_def_corrected_2kp']].dropna()
resSQKPIP = smf.ols(formula='squarekp_ipot_x ~ poten_loans_2income_UZU_corrected_vtor_ + Unmpl_s + ddyprice_def_corrected_2kp - 1', missing='drop', 
                   data=pdfXY5.loc[2009:2019]).fit()
print(resSQKPIP.summary())

                                 OLS Regression Results                                
Dep. Variable:        squarekp_ipot_x   R-squared (uncentered):                   0.993
Model:                            OLS   Adj. R-squared (uncentered):              0.990
Method:                 Least Squares   F-statistic:                              363.3
Date:                Thu, 04 Mar 2021   Prob (F-statistic):                    6.92e-09
Time:                        16:34:03   Log-Likelihood:                         -25.832
No. Observations:                  11   AIC:                                      57.66
Df Residuals:                       8   BIC:                                      58.86
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                                              coef    std err          t      P>|t|      [0.025      0.975]
------------

#### Модель 4

In [247]:
pdfXY4 = demand_ipot.pdfWork.loc[iFirstFactYear:iLastFactYear, ['squareddy_ipot_x','Unmpl_s','loans2income_2pricesratio']].dropna()
resSQDDYIP = smf.ols(formula='squareddy_ipot_x ~ loans2income_2pricesratio + Unmpl_s.shift(1) - 1', missing='drop', 
                   data=pdfXY4.loc[2008:2019]).fit()
print(resSQDDYIP.summary())

                                 OLS Regression Results                                
Dep. Variable:       squareddy_ipot_x   R-squared (uncentered):                   0.921
Model:                            OLS   Adj. R-squared (uncentered):              0.903
Method:                 Least Squares   F-statistic:                              52.26
Date:                Thu, 04 Mar 2021   Prob (F-statistic):                    1.11e-05
Time:                        16:34:03   Log-Likelihood:                         -29.694
No. Observations:                  11   AIC:                                      63.39
Df Residuals:                       9   BIC:                                      64.18
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                                coef    std err          t      P>|t|      [0.025      0.975]
--------------------------

### Расчет результатов (для 4 и 5 моделей + то что с ними увязано)

In [250]:
demand_ipot.pdfWork['loans_MKD_x']=demand_ipot.pdfWork['sdelkiddy_ipot_x']+demand_ipot.pdfWork['sdelkikp_ipot_x']
demand_ipot.pdfWork.loc[iFirstFactYear:iLastFactYear,'loans_IZD_x']=0
demand_ipot.pdfWork.loc[iFirstForecastYear:iLastForecastYear,'loans_IZD_x']=((demand_ipot.pdfWork.loc[iFirstForecastYear:iLastForecastYear,'VvodyIZD']/demand_ipot.pdfWork.loc[iFirstForecastYear:iLastForecastYear,'AvSqIZD'])*demand_ipot.pdfWork.loc[iFirstForecastYear:iLastForecastYear,'s_ipot_inIZDVvody']/100+
                                                                             (demand_ipot.pdfWork.loc[iFirstForecastYear:iLastForecastYear,'AddIZD_x']/demand_ipot.pdfWork.loc[iFirstForecastYear:iLastForecastYear,'AvSqIZD'])*demand_ipot.pdfWork.loc[iFirstForecastYear:iLastForecastYear,'s_ipot_inFPVvody']/100)

demand_ipot.pdfWork['loans_x']=demand_ipot.pdfWork['loans_IZD_x']+demand_ipot.pdfWork['loans_MKD_x']

for i in range (iFirstForecastYear,iLastForecastYear+1):
    demand_ipot.pdfWork.loc[i,'squareddy_ipot_x']=resSQDDYIP.predict(demand_ipot.pdfWork).loc[i]
    demand_ipot.pdfWork.loc[i,'sdelkiddy_ipot_x']=demand_ipot.pdfWork.loc[i,'squareddy_ipot_x']/demand_ipot.pdfWork.loc[i,'AvSqDdy']
    
    demand_ipot.pdfWork.loc[i,'squarekp_ipot_x']=resSQKPIP.predict(demand_ipot.pdfWork).loc[i]
    demand_ipot.pdfWork.loc[i,'sdelkikp_ipot_x']=demand_ipot.pdfWork.loc[i,'squarekp_ipot_x']/demand_ipot.pdfWork.loc[i,'AvSqVtor']
    
    demand_ipot.pdfWork.loc[i,'loans_MKD_x']=demand_ipot.pdfWork.loc[i,'sdelkiddy_ipot_x']+demand_ipot.pdfWork.loc[i,'sdelkikp_ipot_x']
    demand_ipot.pdfWork.loc[i,'loans_x']=demand_ipot.pdfWork.loc[i,'loans_IZD_x']+demand_ipot.pdfWork.loc[i,'loans_MKD_x']
    
    demand_ipot.pdfWork.loc[i,'UZUipot_x']=demand_ipot.pdfWork.loc[i-6:i,'loans_x'].sum()
    
    demand_ipot.pdfWork.loc[i,'UZUipot_s']=demand_ipot.pdfWork.loc[i,'UZUipot_x']/((demand_ipot.pdfWork.loc[i,'Pop_x']/demand_ipot.pdfWork.loc[i,'HHAv'])/1000)
    
    demand_ipot.pdfWork.loc[i+1,'poten_loans_2income_UZU_corrected_perv']=demand_ipot.pdfWork.loc[i+1,'poten_loans_2income_perv']*(1-demand_ipot.pdfWork.loc[i,'UZUipot_s']/demand_ipot.pdfWork.loc[i+1,'_s_family_available_ipot'])
    demand_ipot.pdfWork.loc[i+1,'loans2income_2pricesratio']=demand_ipot.pdfWork.loc[i+1,'poten_loans_2income_UZU_corrected_perv']*10000/demand_ipot.pdfWork.loc[i+1,'ddyprice_def_corrected_2kp']
    
    demand_ipot.pdfWork.loc[i+1,'poten_loans_2income_UZU_corrected_vtor']=demand_ipot.pdfWork.loc[i+1,'poten_loans_2income_vtor']*(1-demand_ipot.pdfWork.loc[i,'UZUipot_s']/demand_ipot.pdfWork.loc[i+1,'_s_family_available_ipot'])
    demand_ipot.pdfWork.loc[i+1,'poten_loans_2income_UZU_corrected_vtor_']=demand_ipot.pdfWork.loc[i+1,'poten_loans_2income_UZU_corrected_vtor']*100
    
demand_ipot.pdfWork['square_ipot_x']=demand_ipot.pdfWork['squarekp_ipot_x']+demand_ipot.pdfWork['squareddy_ipot_x']

### Дополнительные результаты

In [251]:
demand_ipot.pdfWork.loc[iFirstFactYear:iLastFactYear,'loans_vol_IZD']=0
demand_ipot.pdfWork.loc[iFirstForecastYear:iLastForecastYear,'loans_vol_IZD']=0.8*demand_ipot.pdfWork.loc[iFirstForecastYear:iLastForecastYear,'ConstrPriceIZD_x']*demand_ipot.pdfWork.loc[iFirstForecastYear:iLastForecastYear,'loans_IZD_x']


demand_ipot.pdfWork.loc[iFirstForecastYear:iLastForecastYear,'loans_vol_MKD_perv']=demand_ipot.pdfWork.loc[iFirstForecastYear:iLastForecastYear,'sdelkiddy_ipot_x']*demand_ipot.pdfWork.loc[iFirstForecastYear:iLastForecastYear,'loan_vol_av']

demand_ipot.pdfWork.loc[iFirstFactYear:iLastFactYear,'loans_vol_MKD_vtor']=demand_ipot.pdfWork.loc[iFirstFactYear:iLastFactYear,'loans_and_ref_vol_MKD_vtor']/1000000-demand_ipot.pdfWork.loc[iFirstFactYear:iLastFactYear,'refinance_vol']
demand_ipot.pdfWork.loc[iFirstForecastYear:iLastForecastYear,'loans_vol_MKD_vtor']=demand_ipot.pdfWork.loc[iFirstForecastYear:iLastForecastYear,'sdelkikp_ipot_x']*demand_ipot.pdfWork.loc[iFirstForecastYear:iLastForecastYear,'loan_vol_av']

demand_ipot.pdfWork.loc[iFirstFactYear:iLastFactYear,'loans_vol_MKD']=demand_ipot.pdfWork.loc[iFirstFactYear:iLastFactYear,'loans_and_ref_vol_MKD']-demand_ipot.pdfWork.loc[iFirstFactYear:iLastFactYear,'refinance_vol']
demand_ipot.pdfWork.loc[iFirstForecastYear:iLastForecastYear,'loans_vol_MKD']=demand_ipot.pdfWork.loc[iFirstForecastYear:iLastForecastYear,'loans_vol_MKD_perv']+demand_ipot.pdfWork.loc[iFirstForecastYear:iLastForecastYear,'loans_vol_MKD_vtor']

demand_ipot.pdfWork['loans_vol']=demand_ipot.pdfWork['loans_vol_IZD']+demand_ipot.pdfWork['loans_vol_MKD']

### модель №6 "Отношение кредитов, использованных заёмщиками для рефинансирования накопленного ипотечного долга, к прочим выдаваемым кредитам"

In [270]:
demand_ipot.pdfWork['loans4ref2all']=demand_ipot.pdfWork['refinance_vol']/demand_ipot.pdfWork['loans_vol_MKD']*100
demand_ipot.pdfWork['portf_ex_loan_rate']=demand_ipot.pdfWork['portf_rate']-demand_ipot.pdfWork['loan_rate']

demand_ipot.pdfWork['D18']=0
demand_ipot.pdfWork.loc[2018,'D18']=1

pdfXY6 = demand_ipot.pdfWork.loc[iFirstFactYear:iLastFactYear,['loans4ref2all','portf_ex_loan_rate','D18']].dropna()
resLO4RF2A = smf.ols(formula='loans4ref2all ~ portf_ex_loan_rate + D18 -1', missing='drop', 
                    data=pdfXY6.loc[2012:2018]).fit()
print(resLO4RF2A.summary())


                                 OLS Regression Results                                
Dep. Variable:          loans4ref2all   R-squared (uncentered):                   0.870
Model:                            OLS   Adj. R-squared (uncentered):              0.818
Method:                 Least Squares   F-statistic:                              16.69
Date:                Thu, 04 Mar 2021   Prob (F-statistic):                     0.00613
Time:                        17:06:15   Log-Likelihood:                         -14.909
No. Observations:                   7   AIC:                                      33.82
Df Residuals:                       5   BIC:                                      33.71
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                         coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------

### модель №7 "CPR (досрочное погашение)"

In [253]:
demand_ipot.pdfWork['vol_int_payments']=demand_ipot.pdfWork['portf_rate']*((1-(1/demand_ipot.pdfWork['CPIAv'].rolling(5).apply(gmean)**demand_ipot.pdfWork['portf_term']))/(1-1/demand_ipot.pdfWork['CPIAv'].rolling(5).apply(gmean)))

demand_ipot.pdfWork['D17_18']=0
demand_ipot.pdfWork.loc[2017:2018,'D17_18']=1

pdfXY7 = demand_ipot.pdfWork.loc[iFirstFactYear:iLastFactYear,['CPR','vol_int_payments','D15','D17_18']].dropna()
resCPR = smf.ols(formula='CPR ~ vol_int_payments + D15 + D17_18 -1', missing='drop', 
                    data=pdfXY7.loc[2009:2018]).fit()
print(resCPR.summary())

                                 OLS Regression Results                                
Dep. Variable:                    CPR   R-squared (uncentered):                   0.996
Model:                            OLS   Adj. R-squared (uncentered):              0.994
Method:                 Least Squares   F-statistic:                              601.6
Date:                Thu, 04 Mar 2021   Prob (F-statistic):                    8.33e-09
Time:                        16:35:18   Log-Likelihood:                         -11.492
No. Observations:                  10   AIC:                                      28.98
Df Residuals:                       7   BIC:                                      29.89
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                       coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------

### модель №8 "Активы БС"

In [254]:
pdfXY8 = demand_ipot.pdfWork.loc[iFirstFactYear:iLastFactYear,['Activ_x','GDP_x']].dropna()
resACTIV = smf.ols(formula='Activ_x ~ GDP_x -1', missing='drop', 
                    data=pdfXY8.loc[2005:2019]).fit()
print(resACTIV.summary())

                                 OLS Regression Results                                
Dep. Variable:                Activ_x   R-squared (uncentered):                   0.984
Model:                            OLS   Adj. R-squared (uncentered):              0.983
Method:                 Least Squares   F-statistic:                              875.1
Date:                Thu, 04 Mar 2021   Prob (F-statistic):                    5.06e-14
Time:                        16:35:19   Log-Likelihood:                         -51.804
No. Observations:                  15   AIC:                                      105.6
Df Residuals:                      14   BIC:                                      106.3
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [225]:
# demand_ipot.pdfWork[['loan_rate','loans_x','loans_IZD_x','loans_MKD_x','sdelkiddy_ipot_x',
#                     'sdelkikp_ipot_x','loans_vol','loans_vol_IZD','loans_vol_MKD','loans_vol_MKD_perv',
#                     'loans_vol_MKD_vtor','ipot_debt_full','ipot_portf','ipot_portf_to_GDP','ICB',
#                     'portf_rate','portf_term','early_repayment','repayment','refinance_vol','refinance_x',
#                     'outstanding_debt_s','square_ipot_x','squareddy_ipot_x','squarekp_ipot_x']]

In [226]:
demand_ipot.pdfWork[['loan_rate','loans_x','loans_IZD_x','loans_MKD_x','sdelkiddy_ipot_x',
                    'sdelkikp_ipot_x','loans_vol_IZD','loans_vol_MKD','loans_vol_MKD_perv',
                    'loans_vol','portf_rate','portf_term','refinance_vol','ICB',
                    'outstanding_debt_s','squareddy_ipot_x','squarekp_ipot_x']]

,loan_rate,loans_x,loans_IZD_x,loans_MKD_x,sdelkiddy_ipot_x,sdelkikp_ipot_x,loans_vol_IZD,loans_vol_MKD,loans_vol_MKD_perv,loans_vol,portf_rate,portf_term,refinance_vol,ICB,outstanding_debt_s,squareddy_ipot_x,squarekp_ipot_x
date,,,,,,,,,,,,,,,,,
2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005,14.900000,0.141203,0.000000,0.141203,0.032140,0.109063,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.210113,2.111717,NaN
2006,13.700000,0.190030,0.000000,0.190030,0.043254,0.146776,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.042982,2.816116,NaN
2007,12.600000,0.255740,0.000000,0.255740,0.058210,0.197530,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.272760,3.738091,NaN
2008,12.900000,0.344174,0.000000,0.344174,0.059157,0.285017,0.000000,0.655808,0.121000,0.655808,12.900000,17.941667,0.000000,NaN,1.673344,3.741290,14.763884
2009,14.300000,0.128957,0.000000,0.128957,0.012520,0.116437,0.000000,0.152501,0.016000,0.152501,13.178967,16.845357,0.000000,NaN,4.701409,0.772200,5.924064
2010,13.100000,0.298629,0.000000,0.298629,0.028376,0.270253,0.000000,0.379748,0.039000,0.379748,13.161161,16.026515,0.000313,NaN,5.699277,1.705002,13.778887
2011,11.900000,0.516451,0.000000,0.516451,0.072355,0.444096,0.000000,0.716103,0.108000,0.716103,12.612582,14.980855,0.000841,NaN,3.857936,4.206192,22.686387
2012,12.290000,0.676195,0.000000,0.676195,0.126668,0.549527,0.000000,1.025846,0.206000,1.025846,12.461264,14.388784,0.006146,NaN,2.420924,7.147313,28.127839


### Окончательные результаты

In [285]:
demand_ipot.pdfWork.loc[iFirstForecastYear:iLastForecastYear,'loans4ref2all']=resLO4RF2A.predict(demand_ipot.pdfWork).loc[iFirstForecastYear:iLastForecastYear]
demand_ipot.pdfWork.loc[iFirstForecastYear:iLastForecastYear,'CPR'] =         resCPR.predict(demand_ipot.pdfWork).loc[iFirstForecastYear:iLastForecastYear]
demand_ipot.pdfWork.loc[iFirstForecastYear:iLastForecastYear,'Activ_x'] =     resACTIV.predict(demand_ipot.pdfWork).loc[iFirstForecastYear:iLastForecastYear]

demand_ipot.pdfWork.loc[iFirstForecastYear:iLastForecastYear,'refinance_vol']=demand_ipot.pdfWork.loc[iFirstForecastYear:iLastForecastYear,'loans4ref2all']/100*demand_ipot.pdfWork.loc[iFirstForecastYear:iLastForecastYear,'loans_vol']
demand_ipot.pdfWork['refinance_x']=demand_ipot.pdfWork['refinance_vol']/demand_ipot.pdfWork['loan_vol_av']

demand_ipot.pdfWork['ipot_debt_full']=demand_ipot.pdfWork['ipot_debt']+demand_ipot.pdfWork['ipot_debt_prava']
demand_ipot.pdfWork['early_repayment']=(demand_ipot.pdfWork['CPR']*demand_ipot.pdfWork['ipot_debt_full'])/(100-demand_ipot.pdfWork['CPR'])-demand_ipot.pdfWork['refinance_vol']
demand_ipot.pdfWork['ipot_portf']=demand_ipot.pdfWork['ipot_debt_full']+demand_ipot.pdfWork['ICB']

In [286]:
for i in range (iFirstForecastYear,iLastForecastYear+1):
    demand_ipot.pdfWork.loc[i,'ipot_debt_full']=0.09*demand_ipot.pdfWork.loc[i,'Activ_x']
    demand_ipot.pdfWork.loc[i,'early_repayment']=(demand_ipot.pdfWork.loc[i,'CPR']*demand_ipot.pdfWork.loc[i,'ipot_debt_full'])/(100-demand_ipot.pdfWork.loc[i,'CPR'])-demand_ipot.pdfWork.loc[i,'refinance_vol']
    demand_ipot.pdfWork.loc[i,'repayment']=demand_ipot.pdfWork.loc[i-1,'ipot_portf']/demand_ipot.pdfWork.loc[i,'p_MortgLifeAv_x']
    demand_ipot.pdfWork.loc[i,'ipot_portf']=demand_ipot.pdfWork.loc[i-1,'ipot_portf']+demand_ipot.pdfWork.loc[i,'loans_vol']+demand_ipot.pdfWork.loc[i,'refinance_vol']-demand_ipot.pdfWork.loc[i,'repayment']-demand_ipot.pdfWork.loc[i,'early_repayment']

demand_ipot.pdfWork['ICB']=demand_ipot.pdfWork['ipot_portf']-demand_ipot.pdfWork['ipot_debt_full']

In [287]:
demand_ipot.pdfWork[['CPR','loans4ref2all','portf_ex_loan_rate','ICB','ipot_portf','ipot_debt_full','early_repayment','repayment','refinance_vol','refinance_x']]

,CPR,loans4ref2all,portf_ex_loan_rate,ICB,ipot_portf,ipot_debt_full,early_repayment,repayment,refinance_vol,refinance_x
date,,,,,,,,,,
2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008,11.127132,0.000000,0.000000,NaN,NaN,1.164839,0.145841,NaN,0.000000,0.000000
2009,11.014075,0.000000,-1.121033,NaN,NaN,1.119890,0.138612,NaN,0.000000,0.000000
2010,10.625308,0.082529,0.061161,NaN,NaN,1.238508,0.146927,NaN,0.000313,0.000249
2011,12.224180,0.117405,0.712582,NaN,NaN,1.556454,0.215920,NaN,0.000841,0.000614
2012,11.408883,0.599122,0.171264,NaN,NaN,2.078958,0.261585,NaN,0.006146,0.004120
